# Unit 5 - Financial Planning

In [ ]:
# Initial imports
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi

%matplotlib inline

In [ ]:
# Load .env enviroment variables
load_dotenv()

## Part 1 - Personal Finance Planner

### Collect Crypto Prices Using the `requests` Library

In [ ]:
# Set current amount of crypto assets
my_btc = 1.2
my_eth = 5.3

In [ ]:
# Crypto API URLs
btc_url = "https://api.alternative.me/v2/ticker/Bitcoin/?convert=USD"
eth_url = "https://api.alternative.me/v2/ticker/Ethereum/?convert=USD"

In [ ]:
# Fetch current BTC price
BTC_price = requests.get(btc_url)
BTC_content = BTC_price.content
BTC_data = BTC_price.json()
import json
print(json.dumps(BTC_data, indent=4))

In [ ]:
btc_cp = BTC_data["data"]["1"]["quotes"]["USD"]["price"]
print(btc_cp)

In [ ]:
# Fetch current ETH price
ETH_price = requests.get(eth_url)
ETH_content = ETH_price.content
ETH_data = ETH_price.json()
import json
print(json.dumps(ETH_data, indent=4))

In [ ]:
eth_cp = ETH_data["data"]["1027"]["quotes"]["USD"]["price"]
print(eth_cp)

In [ ]:


# Compute current value of my crpto
my_btc_value = my_btc * btc_cp
my_eth_value = my_eth * eth_cp

# Print current crypto wallet balance
print(f"The current value of your {my_btc} BTC is ${my_btc_value:0.2f}")
print(f"The current value of your {my_eth} ETH is ${my_eth_value:0.2f}")

### Collect Investments Data Using Alpaca: `SPY` (stocks) and `AGG` (bonds)

In [ ]:
# Set current amount of shares
my_agg = 200
my_spy = 50

In [ ]:
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")
# Create the Alpaca API object
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")

In [ ]:
# Format current date as ISO format
today = pd.Timestamp("2022-10-21", tz="America/New_York").isoformat()

# Set the tickers
tickers = ["AGG", "SPY"]

# Set timeframe to "1Day" for Alpaca API
timeframe = "1Day"

# Get current closing prices for SPY and AGG
df_portfolio = alpaca.get_bars(
    tickers,
    timeframe,
    start = today,
    end = today
).df
df_portfolio.head()

In [ ]:
# Reorganize the DataFrame
# Separate ticker data
SPY = df_portfolio[df_portfolio['symbol']=='SPY'].drop('symbol', axis = 1)
AGG = df_portfolio[df_portfolio['symbol']=='AGG'].drop('symbol', axis = 1)

# Concatenate the ticker DataFrames
df_portfolio = pd.concat([SPY, AGG],axis=1, keys=['SPY','AGG'])

# Preview DataFrame
df_portfolio.head()

In [ ]:
# Pick AGG and SPY close prices
df_close = pd.DataFrame()
df_close["SPY"] = df_portfolio["SPY"]["close"]
df_close["AGG"] = df_portfolio["AGG"]["close"]
df_close.head()

In [ ]:
df_close.reset_index(drop=True, inplace=True)
df_close

In [ ]:
spy_close_price = df_close._get_value(0, 'SPY')
agg_close_price = df_close._get_value(0, 'AGG')

In [ ]:
# Print AGG and SPY close prices
print(f"Current AGG closing price: ${agg_close_price}")
print(f"Current SPY closing price: ${spy_close_price}")

In [ ]:
# Compute the current value of shares
my_agg_value = agg_close_price * my_agg
my_spy_value = spy_close_price * my_spy

# Print current value of shares
print(f"The current value of your {my_spy} SPY shares is ${my_spy_value:0.2f}")
print(f"The current value of your {my_agg} AGG shares is ${my_agg_value:0.2f}")

### Savings Health Analysis

In [ ]:
# Set monthly household income
monthly_income = 12000

# Consolidate financial assets data
crypto = my_btc_value + my_eth_value
shares = my_agg_value + my_spy_value


# Create savings DataFrame
data = {'amount':[crypto, shares]}
df_savings = pd.DataFrame(data, index= ['crypto', 'shares'])

# Display savings DataFrame
display(df_savings)

In [ ]:
# Plot savings pie chart
df_savings.plot(kind='pie', subplots=True)

In [ ]:
# Set ideal emergency fund
emergency_fund = monthly_income * 3

# Calculate total amount of savings
savings = crypto + shares
savings_needed = emergency_fund - savings
     

In [ ]:
# Validate saving health
if savings > emergency_fund:
    print('congrats, you have enough money in this fund')
elif savings == emergency_fund:
    print('you reached your financial goal') 
elif savings < emergency_fund:
    print(f'you are {savings_needed} away from your goal') 

## Part 2 - Retirement Planning

### Monte Carlo Simulation

In [ ]:
from MCForecastTools import MCSimulation

In [ ]:
api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2"
)

In [ ]:
# Set start and end dates of five years back from today.
# Sample results may vary from the solution based on the time frame chosen
timeframe = "1Day"
start_date = pd.Timestamp('2017-10-01', tz='America/New_York').isoformat()
end_date = pd.Timestamp('2022-10-01', tz='America/New_York').isoformat()

In [ ]:
# Get 5 years' worth of historical data for SPY and AGG
tickers = ["SPY","AGG"]
df_ticker = api.get_bars(
    tickers,
    timeframe,
    start=start_date,
    end=end_date
).df
df_ticker.head()

In [ ]:
# Reorganize the DataFrame
# Separate ticker data
SPY = df_ticker[df_ticker["symbol"]=="SPY"].drop("symbol", axis=1)
AGG = df_ticker[df_ticker["symbol"]=="AGG"].drop("symbol", axis=1)


# Concatenate the ticker DataFrames
df_stock_data = pd.concat([SPY, AGG,], axis=1, keys=["SPY","AGG",])

# Display sample data
df_stock_data.head()

In [ ]:
# Configuring a Monte Carlo simulation to forecast 30 years cumulative returns
MC_60_40 = MCSimulation(
    portfolio_data = df_stock_data,
    weights = [.6,.4,],
    num_simulation = 500,
    num_trading_days = 252*30)

In [ ]:
# Printing the simulation input data
MC_60_40.portfolio_data.head()

In [ ]:
# Running a Monte Carlo simulation to forecast 30 years cumulative returns
MC_60_40.calc_cumulative_return()

In [ ]:
# Plot simulation outcomes
line_plot = MC_60_40.plot_simulation()

In [ ]:
# Plot probability distribution and confidence intervals
dist_plot = MC_60_40.plot_distribution()

### Retirement Analysis

In [ ]:
# Fetch summary statistics from the Monte Carlo simulation results
tbl_ = MC_60_40.summarize_cumulative_return()

# Print summary statistics
print(tbl_)

### Calculate the expected portfolio return at the `95%` lower and upper confidence intervals based on a `$20,000` initial investment.

In [ ]:
# Set initial investment
initial_investment = 20000

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $20,000
ci_lower = round(tbl_[8]*initial_investment,2)
ci_upper = round(tbl_[9]*initial_investment,2)

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 30 years will end within in the range of"
      f" ${ci_lower} and ${ci_upper}")

### Calculate the expected portfolio return at the `95%` lower and upper confidence intervals based on a `50%` increase in the initial investment.

In [ ]:
# Set initial investment
initial_investment_ = 20000 * 1.5

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $30,000
ci_lower_ = round(tbl_[8]*initial_investment_,2)
ci_upper_ = round(tbl_[9]*initial_investment_,2)

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment_} in the portfolio"
      f" over the next 30 years will end within in the range of"
      f" ${ci_lower_} and ${ci_upper_}")

## Optional Challenge - Early Retirement


### Five Years Retirement Option

In [ ]:
# Configuring a Monte Carlo simulation to forecast 5 years cumulative returns with 85 percent in stocks
MC_85_15 = MCSimulation(
    portfolio_data = df_stock_data,
    weights = [.85,.15,],
    num_simulation = 500,
    num_trading_days = 252*5)

In [ ]:
# Running a Monte Carlo simulation to forecast 5 years cumulative returns
MC_85_15.calc_cumulative_return()

In [ ]:
# Plot simulation outcomes
line_plot = MC_85_15.plot_simulation()

In [ ]:
# Plot probability distribution and confidence intervals
dist_plot = MC_85_15.plot_distribution()

In [ ]:
# Fetch summary statistics from the Monte Carlo simulation results
tbl_2 = MC_85_15.summarize_cumulative_return()

# Print summary statistics
print(tbl_2)

In [ ]:
# Set initial investment
investment_2 = 20000

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $60,000
ci_lower_five = round(tbl_2[8]*investment_2,2)
ci_upper_five = round(tbl_2[9]*investment_2,2)

# Print results
print(f"There is a 95% chance that an initial investment of ${investment_2} in the portfolio"
      f" over the next 5 years will end within in the range of"
      f" ${ci_lower_five} and ${ci_upper_five}")

### Ten Years Retirement Option

In [ ]:
# Configuring a Monte Carlo simulation to forecast 10 years cumulative returns
MC_80_20 = MCSimulation(
    portfolio_data = df_stock_data,
    weights = [.8,.2,],
    num_simulation = 500,
    num_trading_days = 252*10)

In [ ]:
# Running a Monte Carlo simulation to forecast 10 years cumulative returns
MC_80_20.calc_cumulative_return()

In [ ]:
# Plot simulation outcomes
line_plot_ = MC_80_20.plot_simulation()

In [ ]:
# Plot probability distribution and confidence intervals
dist_plot_ = MC_80_20.plot_distribution()

In [ ]:
# Fetch summary statistics from the Monte Carlo simulation results
tbl_3 = MC_80_20.summarize_cumulative_return()

# Print summary statistics
print(tbl_3)

In [ ]:
# Set initial investment
investment_3 = 100000

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $60,000
ci_lower_ten = round(tbl_3[8]*investment_3,2)
ci_upper_ten = round(tbl_3[9]*investment_3,2)

# Print results
print(f"There is a 95% chance that an initial investment of ${investment_3} in the portfolio"
      f" over the next 10 years will end within in the range of"
      f" ${ci_lower_ten} and ${ci_upper_ten}")